# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv("city_dataframe.csv")
file.head()


,City ID,City Name,Country,Cloudiness,Max Temp,Wind Speed,Humidity,Lat,Lng
0,0,Punta Arenas,CL,90,57.20,21.85,51,-53.1500,-70.9167
1,1,Almaznyy,RU,75,23.00,20.13,74,48.0448,40.0450
2,2,Cape Town,ZA,0,72.00,9.22,88,-33.9258,18.4232
3,3,Arraial do Cabo,BR,100,76.51,15.68,88,-22.9661,-42.0278
4,4,Yangi Marg`ilon,UZ,100,42.80,4.61,61,40.4272,71.7189


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"]


In [5]:
# heat map
fig = gmaps.figure(center = [0,0], zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                  point_radius = 4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to find your ideal weather condition.
# A max temperature lower than 70 degrees but higher than 60.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_weather_df = file.loc[(file["Max Temp"] >=60) & (file["Max Temp"] <=70) & (file["Wind Speed"] <10) & (file["Cloudiness"] ==0)].dropna()

ideal_weather_df


,City ID,City Name,Country,Cloudiness,Max Temp,Wind Speed,Humidity,Lat,Lng
31,31,Dwārka,IN,0,68.76,8.43,73,22.2394,68.9678
67,67,Salalah,OM,0,68.00,2.30,45,17.0151,54.0924
69,69,Busselton,AU,0,66.00,5.01,71,-33.6500,115.3333
120,120,La Serena,CL,0,64.40,9.22,82,-29.9078,-71.2542
172,172,Beihai,CN,0,63.90,4.81,74,21.4833,109.1000
248,248,Dutse,NG,0,68.04,7.94,23,11.7594,9.3392
295,295,Saldanha,ZA,0,62.60,4.61,88,-33.0117,17.9442
319,319,Sur,OM,0,68.38,5.59,57,22.5667,59.5289
326,326,Tessalit,ML,0,69.04,9.06,19,20.1986,1.0114
349,349,Coquimbo,CL,0,64.40,9.22,82,-29.9533,-71.3436


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df

,City ID,City Name,Country,Cloudiness,Max Temp,Wind Speed,Humidity,Lat,Lng,Hotel Name
0,31,Dwārka,IN,0,68.76,8.43,73,22.2394,68.9678,
1,67,Salalah,OM,0,68.00,2.30,45,17.0151,54.0924,
2,69,Busselton,AU,0,66.00,5.01,71,-33.6500,115.3333,
3,120,La Serena,CL,0,64.40,9.22,82,-29.9078,-71.2542,
4,172,Beihai,CN,0,63.90,4.81,74,21.4833,109.1000,
5,248,Dutse,NG,0,68.04,7.94,23,11.7594,9.3392,
6,295,Saldanha,ZA,0,62.60,4.61,88,-33.0117,17.9442,
7,319,Sur,OM,0,68.38,5.59,57,22.5667,59.5289,
8,326,Tessalit,ML,0,69.04,9.06,19,20.1986,1.0114,
9,349,Coquimbo,CL,0,64.40,9.22,82,-29.9533,-71.3436,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel", 
         "keyword": "hotel", 
         "radius": "5000",
         "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    City_Name = row["City Name"]
    
    params["location"] = f"{lat},{lng}"
    response = requests.get(base_url, params=params).json()
    print(f"Retrieving Results for Index {index}: {City_Name}")
    
    results = response["results"]
    
 
    try:
        print(f"Closest hotel in {City_Name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing result...skipping.")
        
    print("------------------------------------------------------------------------")
  
    time.sleep(1)

Retrieving Results for Index 0: Dwārka
Closest hotel in Dwārka is Lemon Tree Premier.
------------------------------------------------------------------------
Retrieving Results for Index 1: Salalah
Closest hotel in Salalah is HYATT SALALAH HOTEL.
------------------------------------------------------------------------
Retrieving Results for Index 2: Busselton
Closest hotel in Busselton is The Sebel Busselton.
------------------------------------------------------------------------
Retrieving Results for Index 3: La Serena
Closest hotel in La Serena is Enjoy Coquimbo Hotel de la Bahía.
------------------------------------------------------------------------
Retrieving Results for Index 4: Beihai
Closest hotel in Beihai is Shangri-La Hotel, Beihai.
------------------------------------------------------------------------
Retrieving Results for Index 5: Dutse
Closest hotel in Dutse is Bege Guest Inn.
------------------------------------------------------------------------
Retrieving Resul

In [9]:
#updated hotel_df with hotel name
hotel_df

,City ID,City Name,Country,Cloudiness,Max Temp,Wind Speed,Humidity,Lat,Lng,Hotel Name
0,31,Dwārka,IN,0,68.76,8.43,73,22.2394,68.9678,Lemon Tree Premier
1,67,Salalah,OM,0,68.00,2.30,45,17.0151,54.0924,HYATT SALALAH HOTEL
2,69,Busselton,AU,0,66.00,5.01,71,-33.6500,115.3333,The Sebel Busselton
3,120,La Serena,CL,0,64.40,9.22,82,-29.9078,-71.2542,Enjoy Coquimbo Hotel de la Bahía
4,172,Beihai,CN,0,63.90,4.81,74,21.4833,109.1000,"Shangri-La Hotel, Beihai"
5,248,Dutse,NG,0,68.04,7.94,23,11.7594,9.3392,Bege Guest Inn
6,295,Saldanha,ZA,0,62.60,4.61,88,-33.0117,17.9442,Hoedjiesbaai Hotel
7,319,Sur,OM,0,68.38,5.59,57,22.5667,59.5289,Sur Plaza Hotel
8,326,Tessalit,ML,0,69.04,9.06,19,20.1986,1.0114,
9,349,Coquimbo,CL,0,64.40,9.22,82,-29.9533,-71.3436,Enjoy Coquimbo Hotel de la Bahía


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

# reviewed with tutor, had a typo in the code that prevented it from running 

Figure(layout=FigureLayout(height='420px'))